# Phase 4: Fine-tuned 모델로 인기글 생성

Phase 3에서 저장한 파인튜닝 모델을 로드하여 각 게시판별 키워드로 새로운 인기글을 생성합니다.


## ⚙️ 사전 조건
- `refine_training_dataset.ipynb` 실행으로 `outputs/training_dataset_refined.jsonl` 존재
- `phase3_finetune_refined.ipynb` 실행으로 `models_refined/llama3_popular_post_lora` (및 merged_16bit) 존재
- GPU 런타임 할당


---
## 0. 패키지 설치 (필요 시)
- Unsloth/transformers 가 설치되지 않았다면 Phase 3의 설치 셀을 재사용하세요.


In [1]:
# Phase 4에서 사용하는 최소 패키지 설치
# - Unsloth: 파인튜닝된 Llama-3 모델 로드 및 추론용
# - pandas: 생성 결과를 DataFrame/CSV로 저장하기 위해 사용
# - scikit-learn: few-shot 예제 선택을 위한 유사도 계산용

%pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" pandas scikit-learn

print("✅ Phase 4 추론용 패키지 설치 완료")


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.0/288.0 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 153.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 23.6 MB/s eta 0:00:00
✅ Phase 4 추론용 패키지 설치 완료


In [2]:
from google.colab import drive

# Google Drive 마운트 (이미 마운트되어 있다면 건너뛰어도 됩니다)
drive.mount('/content/drive', force_remount=True)

print("✅ Google Drive 마운트 완료")


Mounted at /content/drive
✅ Google Drive 마운트 완료


---
## 1. 경로 설정 및 키워드 로드


In [66]:
from pathlib import Path
import json
import pandas as pd
from IPython.display import display
import re

PROJECT_ROOT = Path("/content/drive/MyDrive/board_crawling")
OUTPUT_DIR = PROJECT_ROOT / "outputs"
MODEL_DIR = PROJECT_ROOT / "models_refined"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)

train_dataset = OUTPUT_DIR / "train_dataset_refined.jsonl"

# 학습 데이터셋 로드 (few-shot 예제 선택용)
training_dataset_path = OUTPUT_DIR / "training_dataset_refined.jsonl"
training_data_parsed = []

if training_dataset_path.exists():
    print(f"✅ 학습 데이터셋 로드 중: {training_dataset_path}")
    with open(training_dataset_path, "r", encoding="utf-8") as f:
        for line in f:
            try:
                entry = json.loads(line)

                # Extract instruction, input_text, response_text directly
                instruction = entry.get("instruction", "").strip()
                input_text = entry.get("input", "").strip()
                response_text = entry.get("output", "").strip()

                # Extract board_name from instruction
                board_name = "Unknown"
                if instruction:
                    match = re.search(r'주어진 키워드를 사용하여 (.*?)의 인기글 스타일로 글을 작성해줘.', instruction)
                    if match:
                        board_name = match.group(1).strip()

                # Extract keywords from input_text
                keywords = []
                if input_text.startswith("키워드:"):
                    keywords_str = input_text[len("키워드:"):].strip()
                    keywords = [kw.strip() for kw in keywords_str.split(',') if kw.strip()]

                training_data_parsed.append({
                    "board_name": board_name,
                    "keywords": keywords,
                    "instruction": instruction,
                    "input_text": input_text,
                    "response_text": response_text
                })
            except Exception as e:
                print(f"⚠️ 라인 파싱 오류: {e}")
                continue
    training_df = pd.DataFrame(training_data_parsed)
    print(f"✅ 학습 데이터셋 로드 완료: {len(training_df)}개 예제")
else:
    print(f"⚠️ {training_dataset_path} 없음. Few-shot 예제 없이 생성합니다.")
    training_df = pd.DataFrame(columns=['board_name', 'keywords', 'instruction', 'input_text', 'response_text'])

# 사용자가 직접 입력할 키워드 딕셔너리
# 예시: "게시판이름": ["키워드1", "키워드2", "키워드3"]
# 이 부분을 직접 수정하여 원하는 게시판 이름과 키워드를 입력하세요.
# 현재는 예시 키워드가 입력되어 있습니다.
trend_keywords = {
    "익게2": ["고양이","vs",
      "더 좋음",
      "강아지",
      "차이",
      "좋음",
      "원래 이렇게"],
    #"자유게시판": ['서버', '과제', '터져서', '안됨', '이번학기', '안되네', '아직', '안돼', '하'],
    "연애상담소": ["여자친구",
      "얼굴",
      "큰거",
      "영화",
      "여자들이",
      "특"],
    #"익게1": ['취업','전공','지원','학기','진로','모집','서강대','오후','관심'],
}

'''
이전 prompt 파일 로딩 로직은 주석 처리되었습니다.
PROMPT_FILES = {
    "익게2": OUTPUT_DIR / "익게2_topics_for_prompt.json",
    "자유게시판": OUTPUT_DIR / "자유게시판_topics_for_prompt.json",
    "연애상담소": OUTPUT_DIR / "연애상담소_topics_for_prompt.json",
    "익게1": OUTPUT_DIR / "익게1_topics_for_prompt.json",
}

trend_keywords = {}
for board, path in PROMPT_FILES.items():
    if not path.exists():
        print(f"⚠️ {board}: {path} 없음 → 건너뜀")
        continue
   with open(path, "r", encoding="utf-8") as f:
        topics_data = json.load(f)
    top_topics = sorted(topics_data, key=lambda x: len(x.get("representatives", [])), reverse=True)[:5]
    keywords = []
    for topic in top_topics:
        kw = topic.get("keywords", [])
        if isinstance(kw, list):
            keywords.extend(kw[:8])
    trend_keywords[board] = list(dict.fromkeys(keywords))[:8]
    print(f"✅ {board}: {len(trend_keywords[board])}개 키워드")
'''

if not trend_keywords:
    raise RuntimeError("생성할 키워드가 없습니다. `trend_keywords` 딕셔너리를 채워주세요.")

print("✅ 사용자가 입력한 키워드를 사용합니다.")
for board, keywords in trend_keywords.items():
    print(f"   - {board}: {', '.join(keywords)}")


✅ 학습 데이터셋 로드 중: /content/drive/MyDrive/board_crawling/outputs/training_dataset_refined.jsonl
✅ 학습 데이터셋 로드 완료: 5752개 예제
✅ 사용자가 입력한 키워드를 사용합니다.
   - 익게2: 고양이, vs, 더 좋음, 강아지, 차이, 좋음, 원래 이렇게
   - 연애상담소: 여자친구, 얼굴, 큰거, 영화, 여자들이, 특


---
## 2. 파인튜닝 모델 로드


In [4]:
from unsloth import FastLanguageModel

FINETUNED_DIR = MODEL_DIR / "llama3_popular_post_lora"
if not FINETUNED_DIR.exists():
    raise FileNotFoundError(f"{FINETUNED_DIR}가 없습니다. Phase 3을 먼저 실행하세요.")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=str(FINETUNED_DIR),
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

FastLanguageModel.for_inference(model)
print("✅ 파인튜닝 모델 로드 완료")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

Unsloth 2025.11.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


✅ 파인튜닝 모델 로드 완료


---
## 3. 생성 함수 정의


In [11]:
import torch
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def split_title_content(text: str):
    """
    생성된 텍스트를 제목과 내용으로 분리합니다.
    """
    lines = text.strip().split("\n")
    if len(lines) > 1:
        title = lines[0].strip()
        content = "\n".join(lines[1:]).strip()
    else:
        title = text[:50].strip() + "..." if len(text) > 50 else text.strip()
        content = text.strip()
    return title, content

def select_few_shot_examples(board_name: str, keywords: list, training_df: pd.DataFrame, num_examples: int = 3):
    """
    학습 데이터에서 키워드 유사도가 높은 few-shot 예제를 선택합니다.
    """
    # 해당 게시판의 데이터만 필터링
    board_df = training_df[training_df['board_name'] == board_name]

    if board_df.empty:
        return ""

    # 입력 키워드를 문자열로 변환
    input_keywords_str = " ".join(keywords)

    # 예제 키워드들을 문자열로 변환
    example_keywords_list = [" ".join(kw_list) for kw_list in board_df['keywords']]

    if not example_keywords_list:
        return ""

    # CountVectorizer로 키워드를 벡터화
    vectorizer = CountVectorizer().fit_transform([input_keywords_str] + example_keywords_list)
    input_vector = vectorizer[0:1]
    example_vectors = vectorizer[1:]

    # 코사인 유사도 계산
    similarities = cosine_similarity(input_vector, example_vectors).flatten()

    # 상위 N개 예제 선택
    top_n_indices = similarities.argsort()[-num_examples:][::-1]

    few_shot_prompt = []
    for idx in top_n_indices:
        example = board_df.iloc[idx]
        example_instruction = f"주어진 키워드를 사용하여 {example['board_name']}의 인기글 스타일로 글을 작성해줘."
        example_input = f"키워드: {', '.join(example['keywords'])}"

        # response_text에서 제목과 내용 분리
        example_title, example_content = split_title_content(example['response_text'])
        example_response = f"제목: {example_title}\n\n내용:\n{example_content}"

        few_shot_prompt.append(
            f"""### Instruction:
{example_instruction}

### Input:
{example_input}

### Response:
{example_response}
"""
        )
    return "\n\n".join(few_shot_prompt)

def generate_post(board_name: str, keywords: list, temperature, max_new_tokens, few_shot_examples: str = ''):
    """
    게시글을 생성합니다. few_shot_examples가 제공되면 이를 프롬프트에 포함합니다.
    """
    instruction = f"주어진 키워드를 사용하여 {board_name}의 인기글 스타일로 글을 작성해줘."

    input_text = f"키워드: {', '.join(keywords)}"

    # 메인 프롬프트 템플릿
    main_prompt_template = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input_text}

### Response:
"""

    # few-shot 예제가 있으면 앞에 추가
    prompt = f"{few_shot_examples}\n\n{main_prompt_template}" if few_shot_examples else main_prompt_template

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=0.9,               # 문맥의 다양성 확보

        # [핵심] 반복을 물리적으로 막는 파라미터들
        repetition_penalty=1.2,  # 단어 반복시 페널티 (1.1 ~ 1.3 추천)
        no_repeat_ngram_size=3,  # 3단어 이상 똑같은 문구가 반복되면 강제 차단

        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=[
            tokenizer.eos_token_id,
            tokenizer.convert_tokens_to_ids("<|eot_id|>") # Llama-3 종료 토큰
        ],
        early_stopping=True
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 결과 파싱
    if "### Response:" in generated_text:
        response = generated_text.split("### Response:")[-1].strip()
    else:
        # 혹시 형식이 깨질 경우 프롬프트 길이만큼 잘라내기
        prompt_len = len(tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True))
        response = generated_text[prompt_len:].strip()

    return response

---
## 4. 게시판별 생성 실행


In [67]:
from datetime import datetime

generated_posts = []
for board, keywords in trend_keywords.items():
    if not keywords:
        continue
    print(f"\n생성 중: {board} ({', '.join(keywords[:])}...)")
    try:
        # Few-shot 예제 선택 (학습 데이터가 있는 경우)
        few_shot_prompt = ""
        if not training_df.empty:
            few_shot_prompt = select_few_shot_examples(board, keywords, training_df, num_examples=5)
            if few_shot_prompt:
                print(f"   📝 Few-shot 예제 {8}개 사용")
            else:
                print(f"   ℹ️ Few-shot 예제 없음 (일반 생성)")

        # 게시글 생성 (few-shot 예제 포함)
        generated_text = generate_post(board, keywords, temperature=0.1, max_new_tokens=200, few_shot_examples=few_shot_prompt)
        title, content = split_title_content(generated_text)
        generated_posts.append({
            "board_name": board,
            "keywords": ", ".join(keywords),
            "generated_title": title,
            "generated_content": content,
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        })
        print(f"✅ {board} 생성 완료")
    except Exception as exc:
        print(f"⚠️ {board} 생성 실패: {exc}")


생성 중: 익게2 (고양이, vs, 더 좋음, 강아지, 차이, 좋음, 원래 이렇게...)
   📝 Few-shot 예제 8개 사용
✅ 익게2 생성 완료

생성 중: 연애상담소 (여자친구, 얼굴, 큰거, 영화, 여자들이, 특...)
   📝 Few-shot 예제 8개 사용
✅ 연애상담소 생성 완료


In [68]:
if generated_posts:
    new_posts_df = pd.DataFrame(generated_posts)
    output_csv = OUTPUT_DIR / "generated_posts.csv"

    if output_csv.exists():
        try:
            existing_df = pd.read_csv(output_csv, encoding="utf-8-sig")
            combined_df = pd.concat([existing_df, new_posts_df], ignore_index=True)
        except pd.errors.EmptyDataError:
            # Handle case where file exists but is empty
            combined_df = new_posts_df
    else:
        combined_df = new_posts_df

    combined_df.to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"\n✅ 생성 결과 저장 완료: {output_csv}")

    print("\n--- 방금 생성된 인기글 ---")
    for index, row in new_posts_df.iterrows():
        print(f"[게시판: {row['board_name']}]")
        print(f"제목: {row['generated_title']}")
        print(f"내용:\n{row['generated_content']}\n")
    print("-----------------------")

    display(combined_df[["board_name", "generated_title","generated_content","timestamp"]])
else:
    print("⚠️ 생성된 게시글이 없습니다.")


✅ 생성 결과 저장 완료: /content/drive/MyDrive/board_crawling/outputs/generated_posts.csv

--- 방금 생성된 인기글 ---
[게시판: 익게2]
제목: 고양이 VS 강아지는 이차이가 좋아서 원래 이런거야?
내용:
https://www.ssodam.com/content/1889097?prev=1&prev_content=/board/3이렇게 생겼던 거였구만...그냥 개랑 다를 게 없어보임…근데 고양이는 뭔가 더 귀엽긴 하더라…개도 귀염한데…? mnohwaqkxhjgqzrhrwvwwlpxbqyfzqoysnqosmofuqotdusnqowcxsqovsqrqoxsqroqsqorqsrqsoqsrqwqswqtrqstqsvqsuqsvtqusvtquvsqvuwqvwqvxqwyqxqwxqxyqyxqyzqzyqzzqzxqzxzqzxqqzxqxxqxxxqxxxxq

[게시판: 연애상담소]
제목: 여자들은 이런건데..
내용:
예전에 여자친군데랑 같이 영화 봤다.근데 여자친굿이랑 다른 여자랑 대화를 했던 부분이 있었음.그래서 나는 그 내용을 듣지는 못했다.그러나 여자친굴이랑 얘기를 끝낸 다음에 "그거 엄청 웃겼어요" 이러길래그냥 궁금해서 물었다."누가 웃으셨나요?" 그러니까 "그 여자들이 서로 대화하면서 웃으신거죠""그럼 누가 무슨 말을 한건가요?" 그러잖아.그랬더니 "그것도 중요하지 않아요. 중요한건 그 사람들이 함께 웃었던 점이에요.""그렇다면 제가 그 상황을 상상하려면 그 두 사람이 무엇을 이야기해야 할까요?" 그러자 "그건 중요하지 않고, 그것들을 기억하려고 노력하시는 것도 의미없습니다. 중요한 것은 그 순간의 감정이죠."이런 식

-----------------------


,board_name,generated_title,generated_content,timestamp
0,익게2,졸업생이 말하는 학점,1. 학생들이 과제 많지? 새내기 학점 잘 받고 싶어서 과제 많이 쓰지 말고 공부만...,2025-12-02 11:54:45
1,익게2,졸업생이 말하는 학점,1. 학생들이 과제 많지? 새내기 학점 잘 받고 싶어서 과제 많이 쓰지 말고 공부만...,2025-12-02 11:54:45
2,익게2,회사 너무 힘들다,나는 이 직장 들어오고나서 처음부터 나 자신이 잘못된 사람이라고 생각했어.성격이 너...,2025-12-02 12:00:54
3,익게2,회사 너무 힘들다,나는 이 직장 들어오고나서 처음부터 나 자신이 잘못된 사람이라고 생각했어.성격이 너...,2025-12-02 12:00:54
4,익게2,회사 너무 힘들다,나는 이 직장 들어오고나서 처음부터 나 자신이 잘못된 사람이라고 생각했어.성격이 너...,2025-12-02 12:00:54
...,...,...,...,...
114,연애상담소,여자들은 이런건데,"예전에 여자친군데랑 같이 영화 봤다.근데 여자친굿이랑 대놓고 얘기했던 내용중에""난 ...",2025-12-06 16:54:58
115,익게2,강아지가랑 고양이가 차이는점,vs좋은데 원래 이런거야?그냥 다들 행복하게 살자!동물도 사람도 다 사랑하자!!+추...,2025-12-06 16:57:33
116,연애상담소,여자들은 이런건데,예전에 여자친군데랑 같이 영화 봤다.근데 여자친굿이랑 나랑은 크게 차이가 나는 부분...,2025-12-06 16:57:49
117,익게2,고양이 VS 강아지는 이차이가 좋아서 원래 이런거야?,https://www.ssodam.com/content/1889097?prev=1&...,2025-12-06 16:58:44


---
## 5. 추가 팁
- 생성 결과를 수작업으로 검토하여 품질 확인 후 배포하세요.
- 다른 샘플을 얻고 싶다면 `temperature`, `max_new_tokens`, 키워드 조합 등을 조정할 수 있습니다.
- 모델을 공유하려면 Phase 3에서 저장한 merged 모델을 업로드하거나 Hugging Face Hub에 push 하면 됩니다.
